In [ ]:
import pickle

import numpy as np
import pandas as pd

In [ ]:
def avg_reward(rewards):
    return np.mean(rewards)

In [ ]:
def dr_success_rate(controlls, tolerances):
    return sum(1 for c, t in zip(controlls, tolerances) if c >= t) / len(tolerances)

In [ ]:
def comfort(controlls, tolerances):
    return np.mean((np.subtract(controlls, tolerances)))

In [ ]:
def saving_energy(olds, controlls, tolerances):
    sum = 0
    for o, c, t in zip(olds, controlls, tolerances):
        sum += (o - c) if c >= t else 0
    
    return sum*(1/4)

In [ ]:
def dr_reward(olds, controlls, tolerances, steps, normal):
    lowest = normal * 0.15
    theta = 24 * 4 * 5 / 2

    dr_theta = steps // 156

    counts = 0
    rewards = []
    records = []
    for o, c, t in zip(olds, controlls, tolerances):
        counts += 1

        if counts > dr_theta - theta:
            records.append(o)

        if counts == dr_theta:
            if c > t:
                cbl = np.mean(sorted(records, reverse=True)[:5])
                saving = cbl - c
                reward = 223.6 * (saving - lowest) * 0.25
                rewards.append(reward if reward > 0 else 0)
            
            counts = 0
            records = []

    return sum(rewards)

In [ ]:
# load info.pkl from multiple dirs

base_path = '/content/drive/My Drive/output/random'
output_dirs = [
    'F_el10_s600_e811_r11_iter45_a25_05_random10-70_a08',
    'F_el10_s600_e811_r11_iter45_a25_05_random10-70_a10',
    'F_el10_s600_e811_r11_iter45_a25_05_random10-70_a12',
    'F_el10_s600_e811_r11_iter45_a25_05_random10-70_a14',
    'F_el10_s600_e811_r11_iter45_a25_05_random10-70_a16',
    'F_el10_s600_e811_r11_iter45_a25_05_random10-70_a20',
    'F_el10_s600_e811_r11_iter45_a25_05_random10-70_a25',
    'F_el10_s600_e811_r11_iter45_a25_05_random10-70_a30',
]

outs = []

for dir in output_dirs:
    with open(f'{base_path}/{dir}/info.pkl', 'rb') as f:
        outs.append(pickle.load(f))

In [ ]:
r = 11

In [ ]:
# DR reward
for out in outs[:]:
    print(dr_reward(out['old_powers'][::r+1], np.add(out['old_powers'][::r+1], out['actions'][::r+1]), out['tolerable_powers'][::r+1], 40500, 0.629317778))

In [ ]:
# Reward
for out in outs:
    print(avg_reward(out['rewards'][::r+1]))

In [ ]:
# DR
for out in outs:
    print(dr_success_rate(np.add(out['old_powers'][::r+1], out['actions'][::r+1]), out['tolerable_powers'][::r+1]))

In [ ]:
# Comfort
for out in outs:
    print(comfort(np.add(out['old_powers'][::r+1], out['actions'][::r+1]), out['tolerable_powers'][::r+1]))

In [ ]:
# Energy
for out in outs:
    print(saving_energy(out['old_powers'][::r+1], np.add(out['old_powers'][::r+1], out['actions'][::r+1]), out['tolerable_powers'][::r+1]))